# Call Center Simulation

## Authors
* Dilara Gökay (2014400102)
* Hatice Kübra Eryılmaz (2014400186)
* Recep Deniz Aksoy (2014400150)

## Description
This assignment shows how a call center can be modeled with a process-interaction view using the [SimPy](https://simpy.readthedocs.io/en/latest/) library.

There are two operators in the system, one is **unskilled** and the other is **expert**. Incoming customers first wait for unskilled operator in a FCFS queue. When they are finished with unskilled operator they wait for expert operator in a FCFS queue. When they are finished with expert operator they leave the system.

In addition to the general flow of the system, expert operator takes break during his/her shift. When the operator decides to take a break, he/she waits until completing all the customers already waiting for her/him. If a new customer arrives after the operator decides to take a break, the operator serves that customer after the break.

The customers are assumed to be extremely patient, as they wait as long as it gets to talk to the operator.

Properties of the system are following:

* In this system, the durations between two consecutive calls are observed to be exponentially distributed with a mean of 15,6 minutes. 
* The time it takes unskilled operator to collect and record the details of a caller is assumed to be distributed according to the Gaussian (i.e. Normal) distribution with a mean of 4,2 and a standard deviation of 1,5 minutes.
* The service time of the expert operator is Exponentially distributed with a mean of 9,3 minutes.
* The expert operator takes 5-min breaks randomly through out the day.
* The number of breaks the expert operator wishes to take during an 8-hour shift is known to be distributed according to a Poisson distribution with a mean of 8 breaks per shift.

## Deliverables
* Following results are printed in the end of the document.
   * server utilization
   * average total waiting time
   * maximum total waiting time to total system time ratio
   * average number of people waiting to be served by the expert operator
* The current number of customers is 1000. This can be changed to 5000 or another number by changing the global variable `NUMBER_OF_CUSTOMERS`
* Current expert service times are distributed exponentially. Please change global variable `EXPERT_DISTRIBUTION` from `exp` to `normal` for  $ServiceTime \sim N(9.3, (3.1)^2)$

## Simulation

In [15]:
import simpy  # for simulation
import random  # for distributions
import numpy as np  # for Poisson distribution

Define a set of globals that define the characteristics of the model instance to be simulated. This includes the seed (RANDOM_SEED) for the random number generators, and key parameters for the interarrival (i.e. mean arrival rate) and service time (i.e. lower and upper bounds for the range) distribution.

In [16]:
RANDOM_SEED = 9780 # for the random number generators
INTERARRIVAL_RATE = 15.6 
NUMBER_OF_CUSTOMERS = 1000
random.seed(RANDOM_SEED)
last_cust_flag = True # used for managing number of breaks
EXPERT_DISTRIBUTION = 'exp'

Define the necessary set of variables for bookkeeping

In [17]:
service_times = []  # duration of the conversation between the customer and the operator (Service time)
queue_w_times = []  # time spent by a customer while it waits for the operator (Queue waiting time Wq)
working_time_unskilled = 0  # total amount of time in which the expert operator is busy
working_time_expert = 0  # total amount of time in which the expert operator is busy
total_waiting_time = [0]  # total waiting time for both operators
total_waiting_time_expert = [0]  # total waiting time for expert operator
max_waiting_time = [0]  # max waiting time for both operators
max_time = 0

* Class for the customers arriving at system. When they are created, they immediatelly initiate a call to unskilled operator (i.e. activate the call process). 
* Once a call is initiated, this is registered as a request to the unskilled operator resource. The customer is put on hold until the resource activates it back. 
* Once the customer is finished with unskilled operator it requests (and waits, if necessary) the expert operator.

In [18]:
class Customer(object):
    global total_waiting_time, max_waiting_time, last_cust_flag, total_waiting_time_expert, max_time
    def __init__(self, name, env, opr):
        self.env = env
        self.name = name
        self.arrival_t = self.env.now
        self.action = env.process(self.call())
    
    
    def call(self):
        global last_cust_flag, max_time
        print('%s initiated a call at %g' % (self.name, self.env.now))
        un_wait = self.env.now
        arrival_expert = 1000000 # the time in which customer is finished with unskilled and starts to wait for expert
        with unskilled_operator.request() as req:
            yield req
            print('%s is assigned to an unskilled_operator at %g' % (self.name, self.env.now))
            un_wait = self.env.now - un_wait
            queue_w_times.append(self.env.now - self.arrival_t)
            total_waiting_time[0]+= self.env.now - self.arrival_t
            yield self.env.process(self.ask_question())
            print('%s is done at unskilled operator on %g' % (self.name, self.env.now))
            exp_wait = self.env.now
            arrival_expert = self.env.now
        with expert_operator.request(priority = 1) as req:
            yield req
            total_waiting_time[0]+= self.env.now - arrival_expert
            total_waiting_time_expert[0]+=self.env.now - arrival_expert
            if self.env.now - self.arrival_t > max_waiting_time[0]:
                max_waiting_time[0] = self.env.now - self.arrival_t
            exp_wait = self.env.now - exp_wait
            print('%s is assigned to an expert operator at %g' % (self.name, self.env.now))
            yield self.env.process(self.ask_question_expert())
            print('%s is done at expert operator on %g' % (self.name, self.env.now))
            wait_total_ratio = (un_wait + exp_wait) / (self.env.now - self.arrival_t)
            if(wait_total_ratio > max_time):
                max_time = wait_total_ratio
            if(self.name[5:] == str((NUMBER_OF_CUSTOMERS - 1))):
                last_cust_flag = False
                
    def ask_question(self):
        global working_time_unskilled
        duration = abs(random.normalvariate(4.2, 1.5))
        print("duration is %s" %duration)
        yield self.env.timeout(duration)
        working_time_unskilled+=duration
        service_times.append(duration)

    def ask_question_expert(self):
        global working_time_expert
        if EXPERT_DISTRIBUTION == 'exp':
            duration = abs(random.expovariate(1/9.3))
        else:
            duration = abs(random.normalvariate(9.3, 3.1))
        yield self.env.timeout(duration)
        working_time_expert+=duration

Break is just like a customer which has the lowest priority among all customers and constant service time(5 min).

In [19]:
class Break(object):
    def __init__(self, name, env, opr):
        self.env = env
        self.name = name
        self.arrival_t = self.env.now
        self.action = env.process(self.call())
        
    def call(self):
        print('A break is initiated at %g' % (self.env.now))
        
        with expert_operator.request(priority = 1000000) as req:
            yield req
            yield self.env.process(self.give_break())
            print('A break is finished at %g' % (self.env.now))
                    
    def give_break(self):
        yield self.env.timeout(5)

In [20]:
def customer_generator(env, operator):
    global end, last_cust_flag
    for i in range(NUMBER_OF_CUSTOMERS):
        yield env.timeout(abs(random.expovariate(1/INTERARRIVAL_RATE)))
        customer = Customer('Cust %s' %(i+1), env, unskilled_operator)

In [21]:
def break_generator(env, operator):
    while(last_cust_flag):
        x = np.random.poisson(1/60.0)
        if(x > 0):
            break_ = Break('Break' , env, operator)
        yield env.timeout(1)

In [22]:
env = simpy.Environment()
unskilled_operator = simpy.Resource(env, capacity = 1)
expert_operator = simpy.PriorityResource(env, capacity = 1)
env.process(customer_generator(env, unskilled_operator))
env.process(break_generator(env, expert_operator))
env.run()
total_time = env.now

Cust 1 initiated a call at 8.61921
Cust 1 is assigned to an unskilled_operator at 8.61921
duration is 6.217751562636504
Cust 2 initiated a call at 13.1948
Cust 1 is done at unskilled operator on 14.837
Cust 1 is assigned to an expert operator at 14.837
Cust 2 is assigned to an unskilled_operator at 14.837
duration is 2.4806954891501443
Cust 2 is done at unskilled operator on 17.3177
Cust 1 is done at expert operator on 18.6759
Cust 2 is assigned to an expert operator at 18.6759
Cust 2 is done at expert operator on 26.0204
Cust 3 initiated a call at 27.4642
Cust 3 is assigned to an unskilled_operator at 27.4642
duration is 3.7400509391032157
Cust 3 is done at unskilled operator on 31.2042
Cust 3 is assigned to an expert operator at 31.2042
Cust 3 is done at expert operator on 44.2056
A break is initiated at 56
A break is finished at 61
A break is initiated at 62
A break is initiated at 65
A break is finished at 67
A break is finished at 72
Cust 4 initiated a call at 93.9065
Cust 4 is as

## Results

In [23]:
print("Server utilization for unskilled operator is %s" %(working_time_unskilled/total_time))

Server utilization for unskilled operator is 0.26711978842205913


In [24]:
print("Server utilization for expert operator is %s" %(working_time_expert/total_time))

Server utilization for expert operator is 0.6174926883712085


In [25]:
print("Average total waiting time for both operators is %s" %(total_waiting_time[0]/NUMBER_OF_CUSTOMERS))

Average total waiting time for both operators is 17.222193774780926


In [26]:
print("Maximum total waiting time to total system time ratio is %s" %(max_time))

Maximum total waiting time to total system time ratio is 0.9809653299888266


In [27]:
print("Average number of people waiting to be served by the expert operator is %s" %(total_waiting_time_expert[0]/total_time))

Average number of people waiting to be served by the expert operator is 1.0270952105307398
